## 爬蟲：捕捉資料
新聞標題的 class (分類) 是根據網頁的原始碼所給定的。

In [ ]:
# 新聞分類：自由時報 資料捕捉一個月_12月有31日
import requests,time,os
import pandas as pd
from bs4 import BeautifulSoup
page=1
df=pd.DataFrame(columns=['title','class','time','link'])
while True:
    time.sleep(3)
    url = "https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page="+str(page)
    print(url)
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.encoding='utf-8'
    if response.status_code!=200:
        break
    else:
        page+=1
    soup=BeautifulSoup(response.text,'html.parser')
    objTag = soup.find_all('div',class_='cont')
    for data in objTag:
        title1=data.find(class_="tit").get_text()
        class1=''
        try:
            class1=data.find(class_="immtag chan").get_text()
        except:
            pass
        try:
            class1=data.find(class_="immtag chan11").get_text()
        except:
            pass   
        time1=data.find('span',class_="time").get_text()
        tag1=data.find('a',class_="tit").get('href')
        df.loc[len(df)]=[title1, class1, time1, tag1]
    file_path = "model_files/ltn1a.csv"  # 29683 data
    write_header = not os.path.exists(file_path)
    df.to_csv(file_path, mode='a', index=False, header=write_header, encoding='utf-8-sig')

https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=1
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=2
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=3
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=4
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=5
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=6
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=7
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=8
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=9
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=10
https://search.ltn.com.tw/list?keyword=AI&start_time=20241201&end_time=20241231&page=11
https://search.ltn.com.tw/list?keyword=AI

## 2. 情感分析
並生成 ltn1_sentiment1.csv

In [ ]:
import pandas as pd
import jieba
df = pd.read_csv("ltn1a.csv")

positive_words = {"好", "提升", "成長", "強", "讚", "創新", "贏", "熱烈", "突破","進補",
"正向","積極","攻頂","觀光","遊憩","合作","智能客服","無人機","揭露"}
negative_words = {"壞", "下滑", "崩盤", "惡化", "危機", "損失", "爭議", "跌", "惱人","過時"
,"震盪","滅絕","暴政","收黑","拖累"}

def simple_sentiment(text):
    tokens = jieba.lcut(str(text))
    pos = sum(1 for w in tokens if w in positive_words)
    neg = sum(1 for w in tokens if w in negative_words)
    if pos > neg:
        return "正面"
    elif neg > pos:
        return "負面"
    else:
        return "中立"
df["sentiment"] = df["title"].apply(simple_sentiment)
df.to_csv("model_files/ltn1_sentiment1.csv", index=False, encoding="utf-8-sig")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yjche\AppData\Local\Temp\jieba.cache
Loading model cost 0.596 seconds.
Prefix dict has been built successfully.


In [4]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
df = pd.read_csv("ltn1a.csv")
if "title" not in df.columns:
    raise ValueError("CSV 檔案中缺少 title 欄位")
classifier = pipeline("text-classification", model="IDEA-CCNL/Erlangshen-RoBERTa-110M-Sentiment")
def analyze_sentiment(text):
    try:
        result = classifier(str(text)[:512])[0] 
        return result["label"]  # 'pos', 'neg', 'neu'
    except:
        return "error"
tqdm.pandas(desc="分析情緒中")
df["sentiment"] = df["title"].progress_apply(analyze_sentiment)
df.to_csv("ltn1_sentiment2.csv", index=False, encoding="utf-8-sig")
print("分析完成，已儲存為 ltn1_sentiment2.csv")


Device set to use cpu
分析情緒中: 100%|██████████████████████████████████████████████████████████████| 29682/29682 [1:31:57<00:00,  5.38it/s]


分析完成，已儲存為 ltn1_sentiment2.csv
